In [14]:
import torch
from transformers import AutoTokenizer, BertForQuestionAnswering

In [15]:
model_path = r"E:\GIT REPOS\medince project\med-buddy\model"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = BertForQuestionAnswering.from_pretrained(model_path, local_files_only=True)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [18]:
question = "What are the uses of Avastin 400mg Injection?"
context = (
    "Medicine Name: Avastin 400mg Injection, Composition: Bevacizumab (400mg), "
    "Uses: Cancer of colon and rectum, lung cancer, "
    "Side Effects: Headache, taste change, rectal bleeding."
)

In [27]:
inputs = tokenizer(question, context, max_length=512, truncation=True, return_tensors="pt")
inputs = {k: v.to(device) for k, v in inputs.items()}

In [28]:
with torch.no_grad():
    outputs = model(**inputs)

In [29]:
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1

In [30]:
answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])
print(f"Predicted Answer: {answer}")

Predicted Answer: 


In [23]:
print("Start Logits:", outputs.start_logits)
print("End Logits:", outputs.end_logits)


Start Logits: tensor([[-0.1499, -0.1050,  0.1103,  0.0493,  0.3222,  0.3714,  0.0520,  0.1403,
          0.4226,  0.5431,  0.5296,  0.5341,  0.1696,  0.0548,  0.3024,  0.4558,
          0.2566,  0.1700,  0.1141,  0.4547,  0.6932,  0.6101,  0.5417,  0.0686,
          0.9289,  0.4252,  0.1260,  0.4536,  0.2801,  0.3606,  0.2495,  0.3393,
          0.5158,  0.5089,  0.1970,  0.0103,  0.5431,  0.2496,  0.2542, -0.1486,
          0.0701,  0.1334, -0.0592,  0.0798,  0.0738,  0.7213,  0.4316,  0.0027,
          0.7830,  0.6705,  0.3746,  0.3940,  0.0643,  0.6962,  0.5311, -0.0060,
          0.0193,  0.2015,  0.4584,  0.2024,  0.1988]])
End Logits: tensor([[ 0.3382,  0.3404,  0.1317,  0.7138,  0.2258,  0.3969, -0.4776, -0.0068,
          0.2653, -0.0376, -0.0650, -0.3285,  0.0165,  0.3406,  0.4519, -0.1019,
          0.1917, -0.5239,  0.0577,  0.2431, -0.1050, -0.0995, -0.5261,  0.2504,
         -0.1272,  0.0108, -0.1609, -0.2845, -0.2267,  0.1251,  0.2155, -0.0015,
          0.0242, -0.3168, 

In [24]:
print("Input IDs:", inputs['input_ids'])
print("Tokens:", tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
print("Attention Mask:", inputs['attention_mask'])

Input IDs: tensor([[  101,  2054,  2024,  1996,  3594,  1997, 10927, 16643,  2078,  4278,
         24798, 13341,  1029,   102,  4200,  2171,  1024, 10927, 16643,  2078,
          4278, 24798, 13341,  1010,  5512,  1024,  2022, 24887, 10993, 12248,
          2497,  1006,  4278, 24798,  1007,  1010,  3594,  1024,  4456,  1997,
         16844,  1998, 28667, 11667,  1010, 11192,  4456,  1010,  2217,  3896,
          1024, 14978,  1010,  5510,  2689,  1010, 28667,  9080,  9524,  1012,
           102]])
Tokens: ['[CLS]', 'what', 'are', 'the', 'uses', 'of', 'ava', '##sti', '##n', '400', '##mg', 'injection', '?', '[SEP]', 'medicine', 'name', ':', 'ava', '##sti', '##n', '400', '##mg', 'injection', ',', 'composition', ':', 'be', '##vac', '##iz', '##uma', '##b', '(', '400', '##mg', ')', ',', 'uses', ':', 'cancer', 'of', 'colon', 'and', 'rec', '##tum', ',', 'lung', 'cancer', ',', 'side', 'effects', ':', 'headache', ',', 'taste', 'change', ',', 'rec', '##tal', 'bleeding', '.', '[SEP]']
Attention Ma

In [31]:
start_idx = torch.argmax(outputs.start_logits, dim=1).item()
end_idx = torch.argmax(outputs.end_logits, dim=1).item() + 1

# Handle cases where start_idx >= end_idx
if start_idx >= end_idx:
    print("Invalid answer range detected.")
    answer = ""
else:
    answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])
print(f"Predicted Answer: {answer}")


Invalid answer range detected.
Predicted Answer: 


In [32]:
print("Max Start Logit:", torch.max(outputs.start_logits))
print("Max End Logit:", torch.max(outputs.end_logits))
print("Min Start Logit:", torch.min(outputs.start_logits))
print("Min End Logit:", torch.min(outputs.end_logits))


Max Start Logit: tensor(0.9289)
Max End Logit: tensor(0.7138)
Min Start Logit: tensor(-0.1499)
Min End Logit: tensor(-0.5261)


In [34]:
import torch.nn.functional as F

start_probs = F.softmax(outputs.start_logits, dim=1)
end_probs = F.softmax(outputs.end_logits, dim=1)

max_start_prob = torch.max(start_probs).item()
max_end_prob = torch.max(end_probs).item()

print(f"Max Start Probability: {max_start_prob}")
print(f"Max End Probability: {max_end_prob}")


Max Start Probability: 0.029939182102680206
Max End Probability: 0.030901720747351646


In [35]:
start_idx = torch.argmax(start_probs, dim=1).item()
end_idx = torch.argmax(end_probs, dim=1).item() + 1

if start_idx < end_idx and max_start_prob > 0.1 and max_end_prob > 0.1:
    answer = tokenizer.decode(inputs['input_ids'][0][start_idx:end_idx])
else:
    answer = "No confident answer found."

print(f"Predicted Answer: {answer}")


Predicted Answer: No confident answer found.
